<a href="https://colab.research.google.com/github/tranquockinh/Fast-Inversion/blob/main/Fast_Inversion_Forward_Calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Earth proflie
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
LayerData = np.array([[150,0],
                      [100,2],
                      [250,5],
                      [400,10],
                      [400,np.inf]])
z = LayerData[:,1]
num_layer = len(z)-1
Vs = LayerData[:,0]
Layer_Vs = Vs[0:-1]
fig,ax = plt.subplots(1,2,figsize=(10,5),dpi=100)
#fig,ax = plt.subplots(figsize=(4,6),dpi=100)
ax[0].step(LayerData[:,0],LayerData[:,1])
ax[0].invert_yaxis()
xlimit = [0,LayerData[-1,0]+50]
#plt.xlim(xlimit[0],xlimit[1])
for i in range(len(LayerData[:,0])-1):
    ax[0].hlines(y=LayerData[i,1],xmin=xlimit[0],xmax=xlimit[1],linestyle='--',linewidth=0.5)
ax[0].set_xlabel('Shear wave velocity, Vs [m/s]')
ax[0].set_ylabel('Depth, z [m]')
ax[0].xaxis.set_label_position('top')
ax[0].xaxis.tick_top()
mid_layer = LayerData[:-1,1]+0.5*(LayerData[1:,1]-LayerData[:-1,1])
print(mid_layer)
label = LayerData[:-1,0]
for i in range(num_layer):
  ax[0].annotate(label[i], (label[i],mid_layer[i]), textcoords='offset points', xytext=(10,0))
halfspace_Vs = ('Halfspace Vs = {}').format(LayerData[-1,0])
ax[0].annotate(halfspace_Vs,(0,LayerData[-2,1]),textcoords='offset points', xytext=(10,-12))

# Wavelength data
Lambda = np.zeros((19))
DeltaLambda = 1
for i in range(len(Lambda)):
    Lambda[0] = 2 # m
    if i > 0:
        Lambda[i] = Lambda[i-1] + DeltaLambda
  # Plot particle motion curve
def Partical_displacement(WL):
    SF=1
    cv = np.array([0.2507, -0.4341, -0.8474*2*np.pi, -0.3933*2*np.pi])
    cv1 = cv[0] 
    cv2 = cv[1] 
    cv3 = cv[2] 
    cv4 = cv[3]
    # Normalization
    zn = z/WL
    # compute amplitude of particle displacements
    pmv = (cv1*np.exp(cv3*zn) + cv2*np.exp(cv4*zn))*SF
    pmv_norm = pmv/pmv[0]
    # plotting
    ax[1].plot(pmv_norm,zn,'-or', markerfacecolor='None')
    ax[1].xaxis.tick_top()
    ax[1].xaxis.set_label_position('top')
    ax[1].set_xlabel('Vertcal displacement amplitude [mm]')
    ax[1].set_ylabel('Normalized depth, zn [m]')
    ax[1].vlines(x=0,ymin=zn[0],ymax=zn[-1],linestyles='--',linewidth=1)
    ax[1].legend(['PR=0.5'],loc='lower right')
    ax[1].invert_yaxis()
    return pmv
pmv = Partical_displacement(Lambda[3])

In [ ]:
def Compute_Weights(WL):
    Ai = np.zeros((num_layer,1),dtype='object')
    wp = np.zeros((num_layer,1),dtype='float')
    SF = 1
    low_bound = z[:-1]
    up_bound = z[1:]
    h = sp.symbols('h')
    cv = np.array([0.2507, -0.4341, -0.8474*2*np.pi, -0.3933*2*np.pi])
    cv1 = cv[0] 
    cv2 = cv[1] 
    cv3 = cv[2] 
    cv4 = cv[3]
    pmv_norm = (cv1*sp.exp(cv3/WL*h) + cv2*sp.exp(cv4/WL*h))*SF 
    for j in range(num_layer):
        Ai[j,0] = sp.integrate(pmv_norm,(h,low_bound[j],up_bound[j]))
        A = sp.integrate(pmv_norm,(h,low_bound[0],up_bound[-1])) 
        wp[j,0] = Ai[j] / A
    return wp
wp = np.zeros((len(Lambda)),dtype='object')
for i in range(len(wp)):
    wp[i] = np.zeros((num_layer))
    wp[i] = Compute_Weights(Lambda[i])
    print(('WAVELENGTH {} PASSED').format(i+1))
    ERROR = 1E-6
    if (np.sum(wp[i]) - 1 > ERROR):
        print('NOT PASSED')
    else: continue

In [ ]:
# Distribution of weighting
labels = ['Layer 1','Layer 2','Layer 3','Layer 4']
fig,ax=plt.subplots(dpi=100)
plt.plot(wp[1],labels)
plt.xlabel('Values of weights, Wp')
plt.ylabel('Layers')
plt.title('Wp distribution')
ax.invert_yaxis()
ax.xaxis.set_label_position('top')
ax.xaxis.tick_top()

In [ ]:
# Compute phase velocity
def Phase_velocity(Layer_Vs,wp):
    Vph = np.dot(Layer_Vs,wp)
    return Vph
Vph = np.zeros((len(Lambda)))
for i in range(len(Lambda)):
    Vph[i] = Phase_velocity(Layer_Vs,wp[i])
print(Vph)
fig,ax=plt.subplots(figsize=(4,6),dpi=100)
ax.plot(Vph,Lambda,'-o',markerfacecolor='None')
ax.invert_yaxis()
ax.set_xlabel('Phase velocity, Vph [m/s]')
ax.xaxis.set_label_position('top')
ax.xaxis.tick_top()
ax.set_ylabel('Wavelength, [m]')